In [1]:
def simulador(t,N,C,Tl,Cmax,List,aci,gci,ab,aco,gco,ae):
#t é um argumento que pode ser 1, 2 ou 3 e corresponde à forma como é formada a solução inicial. t=1 significa que todas as N
#moléculas teem comprimento C. t=2 significa que as N moléculas teem um comprimento no intervalo [0.9*C,1.1*C]. t=3
#significa que a solução inicial é formada pela distribuição de Schulz-Flory
#N=número de moléculas iniciais; C=comprimemto médio das moléculas; Tl=tempo limite
#Cmax=comprimento maximo das moleculas para evaporaçao; List=lista de temperaturas
#aci,gci,ab,aco,gco,ae são as constantes para calcular os tempos médios dos acontecimentos
    
    import moleculas
    import eventos
    import solucao
    import agenda
    import distribuicoes
    import math
    import random
    
    
    def constroisolucao1(N,C):
        s=solucao.novasolucao()
        d=0                    #constroi a solucao inicial com todas as moleculas iniciais com o mesmo comprimento definido 
        while d<N:
            s=solucao.insert(moleculas.novamolecula(C),s)
            d=d+1
        return s
    
    def constroisolucao2(N,C):
        s=solucao.novasolucao()
        d=0
        while d<N:     #constroi a solucao inicial com todos os
            s=solucao.insert(moleculas.novamolecula(random.randint(0.9*C,math.floor(1.1*C))),s)
            d=d+1
        return s
        

    def prob(n, a):   #estas tres funçoes que se seguem teem como funçao criar a solucao inicial da terceira forma
        a=(1-a)       #esta, dado um certo n-tamanho de molécula e um a-usado no enunciado do projeto 
        return 1-(n+1)*a**n+n*a**(n+1)  #devolve a probabilidade acumulada de a molécula ter comprimento n,
                                        #ou seja, de ter menos de ou n carbonos
                                        #o contradominio desta função é [0,1]                            
    def tamanho(a):
        r=random.random()       #esta função, dado o "a" referido no enunciado calcula o tamanho aleartório de uma molécula
        lo,up=1,10**6           #com uma ideia semelhante ao método da bisseção. como intervalos iniciais colocamos 1 e 1000000
        while abs(lo-up)>1:     #pois a probabiliadde de ter mais que 10**6 carbonos é quase nula. e depois dividimos sucessivamente
            mid=math.floor((lo+up)/2) #o intervalo ao meio e verificamos se r (um número pseudo-aleatório entre [0,1]) é maior ou
            p=prob(mid,a)       #menor que essa metade. se for maior, entao a metade agora é o limite inferior do intervalo,
            if p>r:             #se for menor, agora a metade é o limite superior do intervalo, ate que o intervalo esteja  a uma
                up=mid          #distancia de uma unidade. quando isso acontecer encontramos o tamanho da molécula usando a
            else:               #distribuição aleatória dada
                lo=mid
        return lo
    
    def constroisolucao3(N,C):
        s=solucao.novasolucao()  #esta dunção limita-se a dar um valor a a=(2/(C+1)) e a criar a solucao,
        d=0                      #executando "tamanho" um N numero de vezes (N é o numero de moléculas nma solucao)
        while d<N:
            s=solucao.insert(moleculas.novamolecula(tamanho(2/(C+1))),s)
            d=d+1
        return s
        
    
    def modificaagenda(s): #esta função vai ser ativa apenas quando houver mudanças de temperatura no sistema, aí temos que 
        d=0                #reagendar os eventos das moléculas na solucao.
        n=agenda.newa()    #esta função percorre a nossa solução e cria uma nova agenda com os eventos para as moléculas na solucao
        n=agenda.coloca(Ce,n)
        n=agenda.coloca(eventos.evento(Ct+tempoentretemperaturas,"temperatura",[0,0,0]),n)
        while d<solucao.tamanho(s):
            
            n=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)),"cisao",solucao.moleculaposicao(d,s)),n)
            
            if moleculas.ordem(solucao.moleculaposicao(d,s))>0: #so podem sofrer combinação e cisao beta de forem radicais
                n=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aco/gco)*math.e**(-Ctemp/gco)),"comb",solucao.moleculaposicao(d,s)),n)
                n=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ab*(aci/gci)*math.e**(-Ctemp/gci)),"cisaob",solucao.moleculaposicao(d,s)),n)
            
            if moleculas.carbonos(solucao.moleculaposicao(d,s))<=Cmax+1: #so podem sofrer evaporação se tiverem menos de Cmax+1 carbonos
                n=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(solucao.moleculaposicao(d,s))-1)),"evap",solucao.moleculaposicao(d,s)),n)
            d=d+1
        return n
            
    f=open("resultados.txt","w") #abre o "resultados.txt", ficheiro que vai ser usado para criar a animação
    tempofoto = 1                #tempofoto é de quanto em quanto tempo adicionamos uma linha ao "resultados.txt"
    tempoentretemperaturas = Tl / len(List)     #variaveis locais com valores iniciais:
    Ct = 0                                      #tempoentretemperaturas é o tempo entre cada mudança de temperaturas e é calculado                   
                                                #através do quociente entre o tamanho da lista de temperaturas e o tempo limite da simulação
    condensador = solucao.novasolucao()         # s é a nossa solução quente onde estao as moleculas
    dtemperatura = 1                            # condensador é a solucao correspondente ao condensador
    Ctemp = List[0]                             # Ctemp=current temperatura -> temperatura a que o sistema est´1
    numrad = 0                                  # numrad=numero de radicais -> numero de radicais presentes na solucao
    
    if t==1:
        s = constroisolucao1(N,C) #a solucao pode ser formada de 3 maneiras:
    elif t==2:
        s = constroisolucao2(N,C)
    elif t==3:
        s = constroisolucao3(N,C) 
    else:
        return "ERRO! O primeiro argumento tem que ser 1, 2 ou 3"
    
    
    def constroiagendainicial(s):       
        d=0             #constroi a agenda inicial percorrendo a solucao e agendando cisoes a todas a moléculas lá presentes
        a=agenda.newa() #e depois agendando a proxima mudança de temperatura
        while d<len(s):
            a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)),"cisao",s[d]),a)
            d=d+1
        a=agenda.coloca(eventos.evento(Ct+tempoentretemperaturas,"temperatura",[0,0,0]),a)
        return a
    
    a = constroiagendainicial(s)       # a=agenda -> agenda que contem todos os eventos agendados
    Ce=agenda.proximoe(a)              # Ce=current event -> evento sobre o qual vamos atuar, o primeiro evento da agenda
    Ck=eventos.kind(Ce)                # Ck=current kind -> tipo do evento sobre o qual vamos atuar, pode ser "cisao", "cisaob"(cisao-beta)
    Ct=eventos.tempo(Ce)               #"evap"(evaporacao), "comb"(combinacao) ou "temperatura"(mudança de temperatura)
                                       # Ct=current time -> tempo atual do sistema, correspondente ao tempo do evento sobre o qual vamos atuar
    f.write(str(solucao.contagem1(condensador,Cmax+1)) + "\n") #foto inicial-linha escrita no "resultados.txt" com o conteudo do condensador
    
    while Ct < Tl and solucao.tamanho(s)>0:
    #este ciclo controla todo o simulador: a simulação vai acontecer enquanto esta guarda é verdadeira, ou seja, a simulação
    #vai parar apenas quando o tempo do sistema chegar ao tempo limite, ou se por acaso a solução ficar vazia.

        
        if eventos.kind(Ce)=="cisao":  #este ramo calcula o caso em que o proximo evento é uma cisao
            
            if solucao.moleculaQ(eventos.molecula(Ce),s): #só acontecem coisas se a molécula se encontrar na solucao
    #esta clausula é necessária pois mesmo que uma molecula nao esteja na solucao, pode haver eventos relativos a essa molecula
    #na agenda, e quando esses eventos acontecem, nao teem qualquer efeito no sistema pois a molecula alvo do evento nao existe
                

                m1,m2=moleculas.cisao(eventos.molecula(Ce))  #formamos as novas moléculas relativas à cisao da molecula alvo
                s=solucao.remove(eventos.molecula(Ce),s) #é preciso retirar da solucao a molecula que se partiu, pois ja nao existe
                a=agenda.eliminam(eventos.molecula(Ce),a) #e eliminar os eventos agendados para ela (sem ser o atual que vai ser retirado no final)
                s=solucao.insert(m1,s)                       #é precisos inserir na solucao as duas novas moleculas resultantes da cisao
                s=solucao.insert(m2,s)
                
                if moleculas.ordem(eventos.molecula(Ce))==0: 
                    numrad=numrad+2                   #se a molécula que sofreu cisao nao era um radical, forma 2 radicais logo
                else:                                 #o numero de radicais na solucao aumenta por 2. se a molecula que sofreu cisao era
                    numrad=numrad+1                   #um radical, o numero de radicais na solucao aumenta apenas por 1
            
                if moleculas.carbonos(m1)<=3:  #se a molécula formada tiver menos que 4 carbonos, ou seja, menos 3 ligaçoes, evapora
                    condensador=solucao.insert(m1,condensador) #coloca-se no condensador
                    s=solucao.remove(m1,s)  #retira-se da solucao 
                    numrad=numrad-1         #como m1 resultou de um cisao, de certeza que é um radical, logo o numero de radicais
                                            #na solucao diminui por 1
                else:
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)), "cisao",m1),a)
                    #agenda o evento de cisao correspondente a primeira das moleculas resultantes da cisao
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((ab*(aci/gci)*math.e**(-Ctemp/gci))),"cisaob",m1),a)
                    #agenda o evento de cisaobeta  correspondente a primeira das moleculas resultantes da cisao
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aco/gco)*math.e**(-Ctemp/gco)),"comb",m1),a)
                    #agenda o evento de ccombinaçao correspondente a primeira das moleculas resultantes da cisao
                    if moleculas.carbonos(m1)<=Cmax+1:
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(m1)-1)),"evap",m1),a)
                    #se a molecula resultante tem menos de Cmax+1 carbonos vamos agendar a evaporação dessa molecula

                    #repetir o processo para a outra molécula
                if moleculas.carbonos(m2)<=3:
                    condensador=solucao.insert(m2,condensador)
                    s=solucao.remove(m2,s)
                
                    if moleculas.radQ(m2):
                        numrad=numrad-1
                    
                else:
                
                    a = agenda.coloca(eventos.evento(Ct + distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)), "cisao", m2), a)
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ab*((aci/gci)*math.e**(-Ctemp/gci))),"cisaob",m2),a)
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aco/gco)*math.e**(-Ctemp/gco)),"comb",m2),a)
                    if moleculas.carbonos(m2)<=Cmax+1:
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(m2)-1)),"evap",m2),a)
                    
            
            
            
        elif eventos.kind(Ce)=="cisaob":  #realizar as consequencias de quando o evento é cisao-beta
            
            if solucao.moleculaQ(eventos.molecula(Ce),s):
            
                m=moleculas.cisaobeta(eventos.molecula(Ce)) #m é o radical resultante da cisao beta
            
                condensador=solucao.insert(moleculas.novoalceno(),condensador) #sempre que ocorre cisao beta um alceno evapora
            
                s=solucao.insert(m,s)                        #colocar a molecula resultante na solucao
                s=solucao.remove(eventos.molecula(Ce),s)     #retirar a molecula da solucao e
                a=agenda.eliminam(eventos.molecula(Ce),a)    #os eventos relativos a ela da agenda
                
                if moleculas.carbonos(m)<=3:
                    condensador=solucao.insert(m,condensador)  #e agendar os seus eventos
                    s=solucao.remove(m,s)
                
                    if moleculas.radQ(m):
                        numrad=numrad-1
                
                else:
                    
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)),"cisao",m),a)
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ab*(aci/gci)*math.e**(-Ctemp/gci)),"cisaob",m),a)
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aco/gco)*math.e**(-Ctemp/gco)),"comb",m),a)
            
                    if moleculas.carbonos(m)<=Cmax+1:
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(m)-1)),"evap",m),a)

                        
            
        elif eventos.kind(Ce)=="comb":  #consequencias da combinação
            
            if solucao.moleculaQ(eventos.molecula(Ce),s):
                
                if numrad>=2: #combinaçao so acontece se o numero de radicais na solucao for maior que 1
                     
                    r1=solucao.escolherradical(eventos.molecula(Ce),s)  #temos que escolher um radical da solucao, chamado r1
                
                    m=moleculas.combinarad(eventos.molecula(Ce),r1) #e combiná-lo a molecula que sofreu combinaçao
                    s=solucao.insert(m,s)
                    s=solucao.remove(eventos.molecula(Ce),s) #depois retiram-se ambas as moleculas e os eventos relativos a elas
                    a=agenda.eliminam(eventos.molecula(Ce),a)
                    
                    s=solucao.remove(r1,s)
                    a=agenda.eliminam(r1,a)
                
                    if moleculas.radQ(m): #se a molecula resultante da combinaçao for um radical, quer dizer que 2 desapareceram
                        numrad=numrad-1   #e 1 formou-se, logo o numero de radicais na solucao diminui por 1
                    else:                 #se a molecula resultante da combinaçao nao for um radical, diminui por 2
                        numrad=numrad-2
                                          #agendar os eventos da molecula resultante, não é necessário verificar se a molécula
                                          #tem menos de 4 carbonos para evaporar instantaneamente porque é impossivel uma molécula
                                          #que se combinou de outras duas presentes na solucao ter menos de 4 carbonos.
                     
                    if moleculas.radQ(m): #se ela for um radical, agendam-se todos os eventos
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)),"cisao",m),a)
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ab*(aci/gci)*math.e**(-Ctemp/gci)),"cisaob",m),a)
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aco/gco)*math.e**(-Ctemp/gco)),"comb",m),a)
            
                        if moleculas.carbonos(m)<=Cmax+1: #se for pequena o suficiente, pode sofrer evaporação
                            a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(m)-1)),"evap",m),a)
                        
                    else: #se ela nao for um radical nao vai sofrer cisao-beta nem combinaçao
                        a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp((aci/gci)*math.e**(-Ctemp/gci)),"cisao",m),a)
            
                        if moleculas.carbonos(m)<=Cmax+1:
                            a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(m)-1)),"evap",m),a)
                     
            
            
        elif eventos.kind(Ce)=="evap": #consequências da evaporação
            
            if solucao.moleculaQ(eventos.molecula(Ce),s):
            
                if distribuicoes.evapQ(moleculas.carbonos(eventos.molecula(Ce))-1): #verificar se ela se vai evaporar
                    condensador=solucao.insert(eventos.molecula(Ce),condensador) #se sim, entao coloca-la no condensador
                    s=solucao.remove(eventos.molecula(Ce),s)       #e retira-la da solucao e os seus eventos da agenda
                    a=agenda.eliminam(eventos.molecula(Ce),a)
                
                    if moleculas.radQ(eventos.molecula(Ce)): #se a molecula evaporada fosse um radical, entao o numero de radicais
                        numrad=numrad-1                      #na solução diminui por 1
                        
                else:#se ela nao se evaporar, é necessário reagendar um evento de evaporação
                    a=agenda.coloca(eventos.evento(Ct+distribuicoes.tempoexp(ae*(moleculas.carbonos(eventos.molecula(Ce))-1)),"evap",eventos.molecula(Ce)),a)
                    
            
            
            
        elif eventos.kind(Ce)=="temperatura": #evento global de mudança de temperatura:
            Ctemp=List[dtemperatura] #a temperatura muda para a próxima na lista
            dtemperatura=dtemperatura+1 #o índice que percorre a lista das temperaturas aumenta por 1
            a=modificaagenda(s) #e modificamos os eventos agendados para as moléculas na solucao
            
        if eventos.tempo(Ce)>= tempofoto: #este ramo trata do conteúdo do "resultados.txt". quando o tempo do simulador chegar ao
            f.write(str(solucao.contagem1(condensador,Cmax+1)) + "\n") #tempofoto, colocamos o conteudo do simulador no ficheiro
            tempofoto=tempofoto+1 #e incrementamos o tempo para a proxima "foto"
            
            
        
        a=agenda.eliminaproximo(a) #depois de tratar-mos das consequencias do evento, retiramo-lo da agenda
        Ce=agenda.proximoe(a)      #mudamos o eventos que vamos tratar para o proximo evento da aagenda 
        Ck=eventos.kind(Ce)        #mudamos o tipo e tempo do evento 
        Ct=eventos.tempo(Ce)       #e repetimos o ciclo
    
    f.write(str(solucao.contagem1(condensador,Cmax+1)) + "\n") #no final retiramos uma "foto" final colocando o conteudo do simulador
                                                               #no ficheiro "resultados.txt" e fechamo-lo
    f.close()
    return solucao.contagem(condensador,Cmax+1) #no final retornamos o conteúdo do condensador de uma forma mais ilustrativa

In [2]:
%time simulador(1,100,100000,50,20,[200,250,300,350,400,450,500,550,600,650],800,200,100,1600,450,0.1)

Wall time: 1.32 s


[[0, 3],
 [1, 10],
 [2, 7],
 [3, 2],
 [4, 2],
 [5, 1],
 [6, 0],
 [7, 0],
 [8, 0],
 [9, 1],
 [10, 0],
 [11, 0],
 [12, 0],
 [13, 2],
 [14, 0],
 [15, 0],
 [16, 2],
 [17, 0],
 [18, 0],
 [19, 1],
 [20, 1],
 ['alcenos', 17]]

In [3]:
%time simulador(2,100,100000,50,20,[200,250,300,350,400,450,500,550,600,650],800,200,100,1600,450,0.1)

Wall time: 1.84 s


[[0, 26],
 [1, 23],
 [2, 22],
 [3, 6],
 [4, 4],
 [5, 6],
 [6, 1],
 [7, 1],
 [8, 2],
 [9, 3],
 [10, 3],
 [11, 3],
 [12, 1],
 [13, 0],
 [14, 2],
 [15, 1],
 [16, 0],
 [17, 1],
 [18, 1],
 [19, 0],
 [20, 0],
 ['alcenos', 37]]

In [4]:
%time simulador(3,100,100000,50,20,[200,250,300,350,400,450,500,550,600,650],800,200,100,1600,450,0.1)

Wall time: 35.4 s


[[0, 146],
 [1, 154],
 [2, 157],
 [3, 55],
 [4, 37],
 [5, 29],
 [6, 25],
 [7, 18],
 [8, 17],
 [9, 9],
 [10, 11],
 [11, 7],
 [12, 9],
 [13, 5],
 [14, 5],
 [15, 6],
 [16, 4],
 [17, 4],
 [18, 6],
 [19, 2],
 [20, 3],
 ['alcenos', 182]]

In [5]:
%time simulador(4,100,100000,50,20,[200,250,300,350,400,450,500,550,600,650],800,200,100,1600,450,0.1)

Wall time: 998 µs


'ERRO! O primeiro argumento tem que ser 1, 2 ou 3'